In [6]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from system import AtomCavitySystem

def run_simulation(gamma, kappa, beta):
    # Create system
    system = AtomCavitySystem(N=10, g=(0.1 * 2 * np.pi), w0=(1.0 * 2 * np.pi),
                              gamma=gamma, kappa=kappa, beta=beta)

    # Build Hamiltonian
    H = system.create_hamiltonian()

    # Initial state |0⟩_cavity ⊗ |+⟩_atom
    psi11 = system.create_initial_state('0', '+')

    # Collapse operators
    c_ops = system.create_collapse_operators(leaking=True, decay=True, dephasing=True)

    # Expectation values
    e_ops = system.create_expectation_values(
        number_atom=True,
        number_cavity=True,
        coherence_ge=True,
        coherence_eg=True
    )

    # Time evolution
    times = np.linspace(0, 20, 500)
    result = qt.mesolve(H, psi11, times, c_ops, e_ops)

    # Plot
    plt.figure(figsize=(8, 5))
    plt.plot(times, result.expect[0], label="⟨n_atom⟩")
    plt.plot(times, result.expect[1], label="⟨n_cavity⟩")
    plt.plot(times, np.abs(result.expect[2]), label="|coherence|")
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Expectation value")
    plt.grid(True)
    plt.show()

# Create interactive sliders
interact(run_simulation,
         gamma=FloatSlider(min=0, max=0.5, step=0.01, value=0.1, description="Decay γ"),
         kappa=FloatSlider(min=0, max=0.5, step=0.01, value=0.05, description="Leaking κ"),
         beta=FloatSlider(min=0, max=0.5, step=0.01, value=0.02, description="Dephasing β"));


interactive(children=(FloatSlider(value=0.1, description='Decay γ', max=0.5, step=0.01), FloatSlider(value=0.0…